In [1]:
!pip install transformers datasets torch bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 61.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2


In [3]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 7.2 MB/s eta 0:00:00


In [23]:
!pip install --upgrade huggingface_hub

## import libraries

In [4]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

## Huggingface Login

In [25]:
from huggingface_hub import notebook_login
#you need the key from hugging face to authenticate
notebook_login()

## Load dataset and Model

In [5]:
base_model = "meta-llama/Llama-3.2-1B"
# base_model = "google/gemma-2-2b-it"
# base_model = "LoneStriker/Meta-Llama-3-8B-Instruct-4.0bpw-h6-exl2"

In [6]:
from datasets import Dataset
# Dataset.cleanup_cache_files

In [7]:
from codecs import utf_16_encode
# dataset = load_dataset("samsum")
dataset = load_dataset('kalyan003/Question_Answer_Dataset')
dataset = dataset['train']
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Only_Question_answer.txt:   0%|          | 0.00/3.09M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5753 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 5753
})

In [8]:
dataset

Dataset({
    features: ['text'],
    num_rows: 5753
})

In [9]:
# Slice to keep only the last 1000 rows
f_1000_rows = dataset[0:1000]

# Create a new Dataset object from the sliced data
new_dataset = Dataset.from_dict({
    'text': f_1000_rows['text']
})

In [38]:
new_dataset['text'][0]

'<s>[INST] What is the seed lexicon? [/INST] a vocabulary of positive and negative predicates that helps determine the polarity score of an event The seed lexicon consists of positive and negative predicates. If the predicate of an extracted event is in the seed lexicon and does not involve complex phenomena like negation, we assign the corresponding polarity score ($+1$ for positive events and $-1$ for negative events) to the event. We expect the model to automatically learn complex phenomena through label propagation. Based on the availability of scores and the types of discourse relations, we classify the extracted event pairs into the following three types. </s>'

In [39]:
new_dataset['text'][10]

'We used the latest version of the ACP Corpus BIBREF12 for evaluation. It was used for (semi-)supervised training as well. Extracted from Japanese websites using HTML layouts and linguistic patterns, the dataset covered various genres. For example, the following two sentences were labeled positive and negative, respectively:'

In [15]:
compute_dtype = getattr(torch, "float16")

In [26]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
    quantization_type="bitsandbytes_4bit"
)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Unused kwargs: ['quantization_type']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

## traning

In [35]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [69]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    report_to="tensorboard"
)

In [70]:
trainer = SFTTrainer(
    model=model,
    train_dataset=new_dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    # max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params
    # packing=False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [71]:
trainer.train()

Step,Training Loss
25,10.605400
50,8.503700
75,8.037700
100,7.841200
125,7.520900
150,7.727800
175,7.458800
200,7.331200
225,7.124400
250,7.088700


TrainOutput(global_step=10000, training_loss=4.919310472106933, metrics={'train_runtime': 2134.5489, 'train_samples_per_second': 4.685, 'train_steps_per_second': 4.685, 'total_flos': 6248079999959040.0, 'train_loss': 4.919310472106933, 'epoch': 10.0})

## Push to Hub

In [83]:
from huggingface_hub import notebook_login

notebook_login()

In [94]:
# Replace 'your_model_name' with the desired repository name and include your username
repo_name = "osmanh/LLama-3.2-1B-Quantiz-text-generation"
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/osmanh/LLama-3.2-1B-Quantiz-text-generation/commit/2c3aafc8006f81081a49528ec502c16a060450ce', commit_message='Upload tokenizer', commit_description='', oid='2c3aafc8006f81081a49528ec502c16a060450ce', pr_url=None, repo_url=RepoUrl('https://huggingface.co/osmanh/LLama-3.2-1B-Quantiz-text-generation', endpoint='https://huggingface.co', repo_type='model', repo_id='osmanh/LLama-3.2-1B-Quantiz-text-generation'), pr_revision=None, pr_num=None)

## Load and test

In [95]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("osmanh/LLama-3.2-1B-Quantiz-text-generation")
model = AutoModelForCausalLM.from_pretrained("osmanh/LLama-3.2-1B-Quantiz-text-generation")

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Some weights of the model checkpoint at osmanh/LLama-3.2-1B-Quantiz-text-generation were not used when initializing LlamaForCausalLM: ['model.layers.0.self_attn.q_proj.base_layer.weight', 'model.layers.0.self_attn.q_proj.base_layer.weight.absmax', 'model.layers.0.self_attn.q_proj.base_layer.weight.quant_map', 'model.layers.0.self_attn.q_proj.base_layer.weight.quant_state.bitsandbytes__nf4', 'model.layers.0.self_attn.q_proj.lora_A.default.weight', 'model.layers.0.self_attn.q_proj.lora_B.default.weight', 'model.layers.0.self_attn.v_proj.base_layer.weight', 'model.layers.0.self_attn.v_proj.base_layer.weight.absmax', 'model.layers.0.self_attn.v_proj.base_layer.weight.quant_map', 'model.layers.0.self_attn.v_proj.base_layer.weight.quant_state.bitsandbytes__nf4', 'model.layers.0.self_attn.v_proj.lora_A.default.weight', 'model.layers.0.self_attn.v_proj.lora_B.default.weight', 'model.layers.1.self_attn.q_proj.base_layer.weight', 'model.layers.1.self_attn.q_proj.base_layer.weight.absmax', 'model

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [96]:
new_dataset['text'][41]

'<s>[INST] How many questions are in the dataset? [/INST] 2,714 Our dataset is annotated based on Chinese pathology reports provided by the Department of Gastrointestinal Surgery, Ruijin Hospital. It contains 17,833 sentences, 826,987 characters and 2,714 question-answer pairs. All question-answer pairs are annotated and reviewed by four clinicians with three types of questions, namely tumor size, proximal resection margin and distal resection margin. These annotated instances have been partitioned into 1,899 training instances (12,412 sentences) and 815 test instances (5,421 sentences). Each instance has one or several sentences. Detailed statistics of different types of entities are listed in Table TABREF20. </s>'

In [97]:
prompt = "How many questions are in the dataset? "
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<s>[INST] How many questions are in the dataset?  [/INST]ObjectiveObjectiveachsenachsenachsenachsenachsenachsenachsenachsenachsenachsenachsenObjectiveObjectiveObjectiveachsenaberaberaberaberaber nivelponsors}*/

LevelObjectiveObjectiveObjectivelevelLevelplevelplevelplevel nivel nivelfeltObjective Objective :=
 :=
QlevellevelLevelash المهObjectiveObjectiveObjectiveLevelplevelObjectiveObjectiveObjectiveObjectiveObjectiveObjectiveObjectiveObjectiveiniumObjective � �plevelfeltlevellevelObjectiveObjectiveObjectiveObjectiveObjectiveObjectiveObjectiveObjectiveObjectiveObjectiveObjectivelevelわ � � � � �level � � � � � � � � � � � � � � � � �level � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � � �
